In [ ]:
"""
var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End");
    };

    var intervalId = setInterval(clickConnect, 60000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };

    return stopClickConnectHandler;
};

var stopClickConnect = startClickConnect();
"""

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/MyDrive/Study/2024-F/dataset

# !unzip -qq "/content/drive/MyDrive/Study/2024-F/dataset/archive.zip"

/content/drive/MyDrive/Study/2024-F/dataset


In [2]:
import os
import shutil
from PIL import Image
import requests
import torch
from io import BytesIO

from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# 본인 드라이브 경로에 맞게 수정
"""
혜민: split_1
예진: split_2
현진: split_3
성민: split_4
효민: split_5
"""
data_path = "/content/drive/MyDrive/Study/2024-F/dataset/original"

# Data Split

In [ ]:
# dataset load

sub_dirs = ["cats", "dogs"]
cats_files = []
dogs_files = []

for sub_dir in sub_dirs:
    dir_path = os.path.join(data_path, sub_dir)

    if sub_dir == 'cats':
        cats_files = sorted(os.listdir(dir_path))
    else:
        dogs_files = sorted(os.listdir(dir_path))

print("# of cat files: ", len(cats_files))
print("# of dog files: ", len(dogs_files))

# of cat files:  4001
# of dog files:  3439


In [ ]:
output_path = "/content/drive/MyDrive/Study/2024-F/dataset/original"
sub_dirs = ["cats", "dogs"]

def split_and_save_files(data_path, output_path, sub_dir, files, num_splits=5):
    split_size = len(files) // num_splits
    for split_idx in range(num_splits):

        split_folder = os.path.join(output_path, sub_dir, f"split_{split_idx + 1}")
        os.makedirs(split_folder, exist_ok=True)

        start_idx = split_idx * split_size
        end_idx = (split_idx + 1) * split_size if split_idx < num_splits - 1 else len(files)

        for file_name in files[start_idx:end_idx]:
            src_path = os.path.join(data_path, sub_dir, file_name)
            dest_path = os.path.join(split_folder, file_name)
            shutil.copy(src_path, dest_path)

cats_files = sorted(os.listdir(os.path.join(data_path, "cats")))
dogs_files = sorted(os.listdir(os.path.join(data_path, "dogs")))

split_and_save_files(data_path, output_path, "cats", cats_files)
split_and_save_files(data_path, output_path, "dogs", dogs_files)

print("파일 분할 및 저장 완료!")


파일 분할 및 저장 완료!


# Inference


In [3]:
# insruction
instruction = {0: "Make it Christmassy", 1: "Add a santa hat", 2: "Add a rudolph nose", 3: "Make it snowy", 4: "Change to the Christmass mood"}

# Image Guidance Scale
scales = [1.3, 1.5, 1.7]

In [4]:
# Model Load

model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

...

scheduler/scheduler_config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# Inference

# 본인 splilt 번호 확인 후 경로 설정
split_dir = "/content/drive/MyDrive/Study/2024-F/dataset/original/dogs/split_1"
output_dir = "/content/drive/MyDrive/Study/2024-F/dataset/edited image/dogs/split_1"

os.makedirs(output_dir, exist_ok=True)

for file_name in sorted(os.listdir(split_dir)):

    file_path = os.path.join(split_dir, file_name)

    try:
        image = Image.open(file_path)
    except Exception as e:
        print(f"Image load fails: {file_name}, 에러: {e}")
        continue

    for inst_num, inst_text in instruction.items():
        for scale_idx, scale_value in enumerate(scales, start=1):
            generated_images = pipe(
                inst_text,
                image=image,
                image_guidance_scale=scale_value,
                guidance_scale=7.5
            ).images

            # {original file name}_{instruction number}_{scale index}.jpg
            base_name, ext = os.path.splitext(file_name)
            output_name = f"{base_name}_{inst_num}_{scale_idx}.jpg"
            output_path = os.path.join(output_dir, output_name)

            # 이미지 저장
            try:
                generated_images[0].save(output_path)
                print(f"저장 완료: {output_name}")
            except Exception as e:
                print(f"이미지 저장 실패: {output_name}, 에러: {e}")


저장 완료: dog.1047_0_2.jpg


  0%|          | 0/100 [00:00<?, ?it/s]

저장 완료: dog.1047_0_3.jpg


  0%|          | 0/100 [00:00<?, ?it/s]

저장 완료: dog.1047_1_1.jpg


...

  0%|          | 0/100 [00:00<?, ?it/s]

저장 완료: dog.1061_1_1.jpg


  0%|          | 0/100 [00:00<?, ?it/s]

저장 완료: dog.1061_1_2.jpg


  0%|          | 0/100 [00:00<?, ?it/s]